# Python script for data transformation 

## BRICARE:

BRICARE consists of 2 different types of files by year:

a. File after 2022 (2023-2024) = 79 kolom


b. File before 2022 (2019-2022) = 27 kolom


### File Type A


Data Extraction for File Type A must be 2 Files:


A.1 Columns (without "Details")


A.2 Details only 

Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['Create_Date','TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID"

- Columns shoud be mapped based on their Call_Type_ID:

['Produk','Jenis_Produk','Jenis_Laporan']

- PLEASE ADD CIF

#### File A.1 Columns (without "Details"). Please use this if the file is txt

In [4]:
import pandas as pd
import re
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_file(file_path):

    data = []
    date_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}')

    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')

            date_index = next(i for i, part in enumerate(parts) if date_pattern.match(part))

            ticket_id = parts[0] 
            call_type_id = parts[1]  
            description = ';'.join(parts[2:date_index])  
            create_date = parts[date_index]  

      
            data.append([ticket_id, call_type_id, description, create_date] + parts[date_index + 1:])


    df = pd.DataFrame(data, columns=column_names)

    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

    return df


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.txt"

df = parse_file(file_path)
df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
   

df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())



df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")

Data saved to bricare_20230101_20230101.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_17888\1840199040.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('NULL', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_17888\1840199040.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('None', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_17888\1840199040.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with

#### File A.1 Columns (without "Details"). Please use this if the file is csv

In [6]:
import pandas as pd
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')
            if len(parts) > 78:
                description = ';'.join(parts[2:-75])
                new_parts = parts[:2] + [description] + parts[-75:]
                data.append(new_parts)
            else:
                data.append(parts)
    
    df = pd.DataFrame(data, columns=column_names)
    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')
    
    df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
    df.fillna('', inplace=True)
    df = df.replace(['0', 0], '')

    columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
    
    df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())

    return df
file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.csv"
df = parse_csv(file_path)

startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_csv{formatted_startdate}_{formatted_enddate}.csv"
df.to_csv(filename, index=False)

filename


C:\Users\maste\AppData\Local\Temp\ipykernel_17888\3559677339.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_17888\3559677339.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


'bricare_csv20230101_20230101.csv'

#### Cleasing the master Call Type file

In [34]:
import pandas as pd

master_df_path = r"C:\Users\maste\Downloads\bricare\(REVISED) SLA-OLA_NewUserGrouping_Ringkasan Kirim ME Versi 1.6.csv"
df = pd.read_csv(master_df_path, sep=';')


df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')
df = df.dropna(how='all')
df.iloc[:450]
df.to_csv("master_calltype.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_24036\2288781540.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


#### Call type mapping for columns 'Produk', 'Jenis Produk', 'Jenis Laporan'

In [9]:
import pandas as pd


user_dataset_path = r"D:\dataquality\bricare_csv20230101_20230101.csv"
user_df = pd.read_csv(user_dataset_path)
master_df_path = r"D:\dataquality\master_calltype.csv"
master_df = pd.read_csv(master_df_path)


master_df = master_df.rename(columns={
    'Case Types': 'Call_Type_ID', 
    'Product': 'Produk', 
    'Sub Product': 'Jenis_Produk', 
    'Case Category': 'Jenis_Laporan'
})


user_df['Call_Type_ID'] = user_df['Call_Type_ID'].astype(str)
master_df['Call_Type_ID'] = master_df['Call_Type_ID'].astype(str)


merged_df = pd.merge(user_df, master_df[['Call_Type_ID', 'Produk', 'Jenis_Produk', 'Jenis_Laporan']], on='Call_Type_ID', how='left')

user_df['Produk'] = merged_df['Produk_y']
user_df['Jenis_Produk'] = merged_df['Jenis_Produk_y']
user_df['Jenis_Laporan'] = merged_df['Jenis_Laporan_y']

user_df.to_csv(user_dataset_path, index=False)

#### File A.2 Details only. Please use this if the file is txt

In [35]:
import pandas as pd

def process_text_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:  
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]  
                current_entry.append(parts[3].strip())  
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries


def remove_bom_and_strip(df):
    return df.applymap(lambda x: x.replace('\ufeff', '').strip() if isinstance(x, str) else x)


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.txt"
processed_data = process_text_data(file_path)


df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])

if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]

df_final=df_final.iloc[:10]
df_final.iloc[:10].to_csv('details_20230101_20230101.csv', index=False)



#### File A.2 Details only. Please use this if the file is csv

In [14]:
import pandas as pd

def process_csv_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]
                current_entry.append(parts[3].strip())
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries

file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.csv"
processed_data = process_csv_data(file_path)

df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])


if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]


df_final = df_final.iloc[:10]
output_path = "details_csv20230101_20230101.csv"
df_final.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to details_csv20230101_20230101.csv


#### Merge the file A.1 and file A.2

In [38]:
import pandas as pd

#just take 10 lines for an example
path=r"D:\dataquality\bricare_20230101_20230101.csv"
df=pd.read_csv(path)
df.iloc[:10].to_csv(path,index=False)

In [17]:
import pandas as pd


file_path_1 = r"D:\dataquality\bricare_csv20230101_20230101.csv"
file_path_2 = r"D:\dataquality\details_csv20230101_20230101.csv"


df_tenline_bricare = pd.read_csv(file_path_1)
df_detail_bricare_10line = pd.read_csv(file_path_2)

df_detail_bricare_10line.columns = ['Ticket_ID', 'Details']

merged_df = pd.merge(df_tenline_bricare, df_detail_bricare_10line, on='Ticket_ID', how='left')


output_file_path = r"D:\dataquality\bricare_csv20230101_20230101.csv"


column_to_move="Details"
merged_df = merged_df[[col for col in merged_df if col != column_to_move][:3] + [column_to_move] + [col for col in merged_df if col != column_to_move][3:]] 

merged_df.to_csv(output_file_path, index=False)

### File Type B


Data Extraction for File Type B (27 columns)


Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['TanggalClosed', 'tanggalTransaksi','Create_Date']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID" if any

- PLEASE ADD CIF



In [52]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]


path = r"C:\Users\maste\Downloads\BRICARE_25042024 masking.csv"


data = pd.read_csv(path, delimiter=';')


data['Column1'] = data['Column1'].astype(str)
data_cleaned = data[data['Column1'].str.match(r'TTB\d+')]

data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)
data_cleaned = data_cleaned[data_cleaned['Column2'].str.match(r'^\d{4}$')]

data_cleaned['Column4'] = pd.to_datetime(data_cleaned['Column4'], errors='coerce')
data_cleaned = data_cleaned.dropna(subset=['Column4'])


data_to_drop = ['Column28', 'Column29', 'Column30', 'Column31', 'Column32']
data_cleaned = data_cleaned.drop(columns=data_to_drop)


if len(data_cleaned.columns) <= len(column_list):
    data_cleaned.columns = column_list[:len(data_cleaned.columns)]


data_cleaned.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
data_cleaned.fillna('', inplace=True)
data_cleaned = data_cleaned.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    data_cleaned[column] = pd.to_datetime(data_cleaned[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    data_cleaned[column] = data_cleaned[column].apply(lambda x: '' if pd.isna(x) else x)

# Just take 10 lines for an example
data_cleaned = data_cleaned.iloc[:10]


data_cleaned.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(data_cleaned['Create_Date']))
enddate = pd.Timestamp(max(data_cleaned['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


data_cleaned.to_csv(filename, index=False)

print(f"Data saved to {filename}")


C:\Users\maste\AppData\Local\Temp\ipykernel_33500\3452762791.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)


Data saved to bricare_20200101_20200101.csv


## Check the number of columns

In [55]:

path1=r"D:\dataquality\bricare_20230101_20230101.csv"
path2=r"D:\dataquality\bricare_20200101_20200101.csv"

df1=pd.read_csv(path1)
df2=pd.read_csv(path2)
print(len(df1.columns))
print(len(df2.columns))

print(df1.columns)
print(df2.columns)

79
27
Index(['Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Details', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name', 'Jenis_Produk', 'Last_Modified_By', 'Merchant_ID',
       'Modified_Date', 'NOTAS', 'Produk', 'SLA_Status', 'TID',
       'tanggalAttachmentDone', 'Tgl_Assigned', 'Tgl_Eskalasi', 'AnalisaSkils',
       'Attachment_', 'Bank_BRI', 'Biaya_Admin', 'Suku_Bunga', 'Bunga',
       'Butuh_Attachment', 'Cicilan', 'Hasil_Kunjungan', 'Log_Name',
       'MMS_Ticket_Id', 'Mass_Ticket_Upload_Flag', 'Nama_Supervisor',
       'Nama_TL', 'Nama_Wakabag', 'Nasabah_Prioritas', 'Notify_By',
       'Organization', 'Output_Settlement', 'phone_

## To Compare two files

In [18]:
# Load the newly uploaded files for detailed comparison
processed_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"
expected_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"

# Read the files
processed_df_newest = pd.read_csv(processed_file_path_newest)
expected_df_newest = pd.read_csv(expected_file_path_newest)

# Check for exact match first
exact_match = processed_df_newest.equals(expected_df_newest)

# Initialize a dictionary to store detailed comparison results
comparison_details = {
    'shape_match': processed_df_newest.shape == expected_df_newest.shape,
    'columns_match': processed_df_newest.columns.equals(expected_df_newest.columns),
    'column_differences': {},
    'value_differences': {}
}

# Compare each aspect if exact match is not true
if not exact_match:
    # Check shape
    if not comparison_details['shape_match']:
        comparison_details['shape_details'] = {
            'processed_shape': processed_df_newest.shape,
            'expected_shape': expected_df_newest.shape
        }
    
    # Check columns
    if not comparison_details['columns_match']:
        comparison_details['column_details'] = {
            'processed_columns': processed_df_newest.columns.tolist(),
            'expected_columns': expected_df_newest.columns.tolist()
        }

    # Check column-by-column values
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            comparison_details['column_differences'][column] = processed_df_newest[column].compare(expected_df_newest[column])

# Summarize value differences
if not comparison_details['columns_match']:
    value_differences = {}
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            value_differences[column] = processed_df_newest[column].compare(expected_df_newest[column])
    comparison_details['value_differences'] = value_differences

comparison_details


{'shape_match': True,
 'columns_match': True,
 'column_differences': {},
 'value_differences': {}}

# To collect all Error logs in a path

In [37]:
import pandas as pd

path=r"D:\dataquality\bricare_20230101_20230101.csv"
df=pd.read_csv(path)
df=df.iloc[:4]
# df.to_csv("bricare_test20230101_20230101.csv", index=False)
path2= r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error_logs.csv"
df2=pd.read_csv(path2)
df2

,TICKET_ID,CALL_TYPE_ID,CALL_TYPE,DETAILS,CREATE_DATE,GATEWAY,JENIS_LAPORAN,NAMA_NASABAH,NO_REKENING,NOMINAL,...,TGL_FOWARD,TGL_IN_PROGRESS,TGL_RETURNED,TICKET_REFERENSI,TIKET_URGENCY,TIPE_REMARK,UNIQUEID,USERS,USERGROUP_ID,ERROR
0,TTB000043833835,8701,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,2023-01-01 07:07:15,Phone,Request,Anwar,123456789012345,0.0,...,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4,Account ID: id value of incorrect type: TTB000...
1,TTB000043833951,8202,Informasi Product Banking,#BRILINKMOB\n\nDATA outlet BRILINK\nKode Outle...,2023-01-01 07:06:39,Phone,Products / Promotion Inquiry,Budi,123456789012345,0.0,...,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4,Account ID: id value of incorrect type: TTB000...
2,TTB000043833734,8701,Blokir Kartu ATM karena kartu hilang,"#CALL TERPUTUS\n\nif ch call back ,layanan IB ...",2023-01-01 07:05:16,Phone,Request,Ceshy,123456789012345,0.0,...,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4,Account ID: id value of incorrect type: TTB000...
3,TTB000043833965,8413,Nasabah BRI gagal tarik tunai & terdebet di AT...,Nasabah gagal melakukan transaksi tarik tunai ...,2023-01-01 07:04:46,Phone,Complaint - Transaction,Dion Fury,123456789012345,2500000.0,...,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,1,Account ID: id value of incorrect type: TTB000...


In [23]:
import os
import pandas as pd

def combine_error_logs(directory_path):
    combined_df = pd.DataFrame()


    for filename in os.listdir(directory_path):
        if filename.startswith("error"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    

    combined_df = combined_df.drop_duplicates()
   


    output_path = os.path.join(directory_path, 'error_logs.csv')
    combined_df.to_csv(output_path, index=False)
    
    print(f"Combined error logs saved to: {output_path}")


directory_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log"
combine_error_logs(directory_path)


Combined error logs saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\log\error_logs.csv


In [34]:
import os
import pandas as pd

def combine_error_logs(directory_path):
    combined_df = pd.DataFrame()

    for filename in os.listdir(directory_path):
        if filename.startswith("error"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Normalize the 'TICKET_ID' to ensure duplicates are identified
    combined_df['TICKET_ID'] = combined_df['TICKET_ID'].str.upper()

    # Remove duplicates based on the normalized 'TICKET_ID' column
    combined_df = combined_df.drop_duplicates(subset='TICKET_ID')

    output_path = os.path.join(directory_path, 'error_logs.csv')
    combined_df.to_csv(output_path, index=False)
    
    return output_path, combined_df

directory_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log"
output_path, combined_df = combine_error_logs(directory_path)

output_path


'C:\\Users\\maste\\Downloads\\dataloader_v60.0.2\\log\\error_logs.csv'